<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/amazon/NB_unknownwords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

Check whether stemming & lemmatization make difference

Do we need bigram/trigram frequencies (within titles)?

Do we need dataframe including cleaned data & genres? (which cleaned data?) -> create dictionary of lists maybe
https://www.geeksforgeeks.org/python-ways-to-create-a-dictionary-of-lists/

How do we incorporate POS-tagging & NER in classifier?

# Importing & Cleaning data

This section imports the data into a pandas dataframe and goes through the following preprocessing steps:

-Case collapsing 
-Remove punctuation 
-Tokenization 
-N-Grams: bigrams and trigrams 
-Stemming -> check whether this makes a difference 
-Lemmatization -> check whether this makes a difference 
-Part-of-speech (POS) tagging 
-Named entity recognition (NER) 

## Import Data
We first import the data as a pandas dataframe. Then we create a list made up of the title and genre columns from the original data. We also create a list including all 32 genres.

In order to import the data into Google Colab, we have to first upload the csv file:

In [ ]:
# mount Google Drive
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
# import file from Google Drive
import pandas as pd
df = pd.read_csv('/drive/My Drive/book32listing.csv',encoding='latin1', header=None)

In [ ]:
# drop columns that are not needed
#df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6]] # only columns with titles and genres
df1.columns = ['title', 'genre']
print(df1)

                                                    title      genre
0                         Mom's Family Wall Calendar 2016  Calendars
1                         Doug the Pug 2016 Wall Calendar  Calendars
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars
4                    Sierra Club Engagement Calendar 2016  Calendars
...                                                   ...        ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel

[207572 rows x 2 columns]


In [ ]:
titles = df1['title'] # list of all titles
titles1 = titles.values.tolist() # change to list of strings
print(titles1[0:6]) # test whether it worked

["Mom's Family Wall Calendar 2016", 'Doug the Pug 2016 Wall Calendar', 'Moleskine 2016 Weekly Notebook, 12M, Large, Black, Soft Cover (5 x 8.25)', '365 Cats Color Page-A-Day Calendar 2016', 'Sierra Club Engagement Calendar 2016', 'Sierra Club Wilderness Calendar 2016']


In [ ]:
genres = df1['genre']
genres = genres.values.tolist()
genres = pd.DataFrame(genres)

In [ ]:
df1.genre.unique() # list of all possible genres

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

## Case Collapsing
Change all uppercase to lowercase letters

In [ ]:
case_collap = map(lambda x:x.lower(), titles1)
case_collap_list = list(case_collap)
print(case_collap_list[0:6])

["mom's family wall calendar 2016", 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook, 12m, large, black, soft cover (5 x 8.25)', '365 cats color page-a-day calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Remove Punctuation
Remove punctuation by creating a translation table. 
Punctuation to be removed is given in string: string.punctuation

In [ ]:
import string
trans = str.maketrans('', '', string.punctuation)
rem_punct = [s.translate(trans) for s in case_collap_list]
print(rem_punct[0:6])

['moms family wall calendar 2016', 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook 12m large black soft cover 5 x 825', '365 cats color pageaday calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Tokenization
This splits all titles into words (output: list of lists of strings)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokenized_titles = [word_tokenize(i) for i in rem_punct]
print(tokenized_titles[:10])

[['moms', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cats', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016'], ['thomas', 'kinkade', 'the', 'disney', 'dreams', 'collection', '2016', 'wall', 'calendar'], ['ansel', 'adams', '2016', 'wall', 'calendar'], ['dilbert', '2016', 'daytoday', 'calendar'], ['mary', 'engelbreit', '2016', 'deluxe', 'wall', 'calendar', 'never', 'give', 'up']]


## N-Grams: Bigrams and Trigrams
Create bigrams + trigrams (could be done in one go e.g. n=2 for bigrams, n=3 for trigrams etc.)

In [ ]:
# bigrams
token_bigram = []
for title in tokenized_titles:
    title_bigram = []
    for w in range(len(title) - 1):
        title_bigram.append([title[w], title[w + 1]])
    token_bigram.append(title_bigram)
print(token_bigram[:6]) # test whether working

[[['moms', 'family'], ['family', 'wall'], ['wall', 'calendar'], ['calendar', '2016']], [['doug', 'the'], ['the', 'pug'], ['pug', '2016'], ['2016', 'wall'], ['wall', 'calendar']], [['moleskine', '2016'], ['2016', 'weekly'], ['weekly', 'notebook'], ['notebook', '12m'], ['12m', 'large'], ['large', 'black'], ['black', 'soft'], ['soft', 'cover'], ['cover', '5'], ['5', 'x'], ['x', '825']], [['365', 'cats'], ['cats', 'color'], ['color', 'pageaday'], ['pageaday', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'engagement'], ['engagement', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'wilderness'], ['wilderness', 'calendar'], ['calendar', '2016']]]


In [ ]:
# trigrams
token_trigram = []
for title in tokenized_titles:
    title_trigram = []
    for w in range(len(title) - 2):
        title_trigram.append([title[w], title[w + 1], title[w + 2]])
    token_trigram.append(title_trigram)
print(token_trigram[:6]) # test whether working

[[['moms', 'family', 'wall'], ['family', 'wall', 'calendar'], ['wall', 'calendar', '2016']], [['doug', 'the', 'pug'], ['the', 'pug', '2016'], ['pug', '2016', 'wall'], ['2016', 'wall', 'calendar']], [['moleskine', '2016', 'weekly'], ['2016', 'weekly', 'notebook'], ['weekly', 'notebook', '12m'], ['notebook', '12m', 'large'], ['12m', 'large', 'black'], ['large', 'black', 'soft'], ['black', 'soft', 'cover'], ['soft', 'cover', '5'], ['cover', '5', 'x'], ['5', 'x', '825']], [['365', 'cats', 'color'], ['cats', 'color', 'pageaday'], ['color', 'pageaday', 'calendar'], ['pageaday', 'calendar', '2016']], [['sierra', 'club', 'engagement'], ['club', 'engagement', 'calendar'], ['engagement', 'calendar', '2016']], [['sierra', 'club', 'wilderness'], ['club', 'wilderness', 'calendar'], ['wilderness', 'calendar', '2016']]]


## Stemming
Test this out to see whether it makes a difference in final classifier

PorterStemmer (one algorithm for stemming; less aggressive than LancasterStemming)

We first create an empty list to contain lists of stems in each title. The second empty list will contain the stem of the title. Each stemmed word in the title will be added to the second list. Each time the second list is filled, it will appended as a list to the first list. 

In [ ]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
stems = []   
for title in tokenized_titles:
    stems_title = []
    for word in title:
        stems_title.append(porter.stem(word))
    stems.append(stems_title)
    
print(stems[0:6]) # test whether it works

[['mom', 'famili', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskin', '2016', 'weekli', 'notebook', '12m', 'larg', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engag', 'calendar', '2016'], ['sierra', 'club', 'wilder', 'calendar', '2016']]


## Lemmatization
Same principle as with stemming. 
We should test this out to see whether it makes a difference in final classifier

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmas = []
for title in tokenized_titles:
    lemmas_title = []
    for word in title:
        lemmas_title.append(lemmatizer.lemmatize(word))
    lemmas.append(lemmas_title)
    
print(lemmas[0:6]) # test whether it works

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['mom', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016']]


## Part-of-Speech (POS) Tagging
Create list of lists with tokens and their corresponding part-of-speech tag in each title

In [ ]:
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

postag = []
for title in tokenized_titles:
    postag.append(nltk.pos_tag(title))
    
print(postag[0:6]) # testing whether postag worked

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[[('moms', 'NNS'), ('family', 'NN'), ('wall', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('doug', 'VB'), ('the', 'DT'), ('pug', 'NN'), ('2016', 'CD'), ('wall', 'NN'), ('calendar', 'NN')], [('moleskine', 'NN'), ('2016', 'CD'), ('weekly', 'JJ'), ('notebook', 'NN'), ('12m', 'CD'), ('large', 'JJ'), ('black', 'JJ'), ('soft', 'JJ'), ('cover', 'NN'), ('5', 'CD'), ('x', 'JJ'), ('825', 'CD')], [('365', 'CD'), ('cats', 'NNS'), ('color', 'VBP'), ('pageaday', 'IN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('engagement', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('wilderness', 'NN'), ('calendar', 'NN'), ('2016', 'CD')]]


## Named Entity Recognition (NER)
Create list of lists with words, correspondent POS and named entity tags for each title. This uses postags created in previous step

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk
from nltk.chunk import tree2conlltags

ner = []
for title in titles1:
    ner.append(tree2conlltags(ne_chunk(pos_tag(word_tokenize(title)))))
    
print(ner[0:6]) # test whether NER worked

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[[('Mom', 'NNP', 'B-PERSON'), ("'s", 'POS', 'O'), ('Family', 'NNP', 'B-PERSON'), ('Wall', 'NNP', 'I-PERSON'), ('Calendar', 'NNP', 'I-PERSON'), ('2016', 'CD', 'O')], [('Doug', 'NNP', 'O'), ('the', 'DT', 'O'), ('Pug', 'NNP', 'O'), ('2016', 'CD', 'O'), ('Wall', 'NNP', 'B-FACILITY'), ('Calendar', 'NNP', 'I-FACILITY')], [('Moleskine', 'NN', 'O'), ('2016', 'CD', 'O'), ('Weekly', 'NNP', 'O'), ('Notebook', 'NNP', 'O'), (',', ',', 'O'), ('12M', 'CD', 'O'), (',', ',', 'O'), ('Large', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Black', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Soft', 'NNP', 'B-PERSON'), ('Cover', 'NNP', 'I-PERSON'), ('(', '(', 'O'), ('5', 'CD', 'O'), ('x', 'RB', 'O'), ('8.25', 'CD', 'O'), (')', ')', 'O')], [('365', 'CD', 'O'), ('Cats', 'NNPS', '

# Classifiers
Check whether stemming/lemmatization make difference in final classifiers
-> does it improve/worsen classifier?

## Split data into train/test

In order to split the data into train and test sets, we create a dataframe containing the tokenized titles. We then split the data into 80% for training and 20% for testing. 

In [ ]:
# create dataframe containing tokenized titles and genres
tok_title = pd.DataFrame({0: tokenized_titles})
data_in = [tok_title[0], df1["genre"]]
headers = ["titles", "genres"]

data = pd.concat(data_in, axis=1, keys=headers)
print(data[:5])

                                              titles     genres
0               [moms, family, wall, calendar, 2016]  Calendars
1             [doug, the, pug, 2016, wall, calendar]  Calendars
2  [moleskine, 2016, weekly, notebook, 12m, large...  Calendars
3       [365, cats, color, pageaday, calendar, 2016]  Calendars
4         [sierra, club, engagement, calendar, 2016]  Calendars


In [ ]:
# split data into train and test
import numpy as np

test_pct=0.2 # split into 80/20%

# create mask
mask = np.random.choice([0, 1], p=[1 - test_pct, test_pct], size=data.shape[0])

# apply mask
data["mask"] = mask
test = data[data["mask"] == 1]
train = data[data["mask"] == 0]

# removing column
test = test.drop("mask", axis="columns").reset_index()
train = train.drop("mask", axis="columns").reset_index()

# remove original indexing data (otherwise we have double indexing)
test = test.drop("index", axis="columns")
train = train.drop("index", axis="columns")

In [ ]:
# save split datasets as csv files in Google Drive
test.to_csv('/drive/My Drive/test.csv', index=False)
train.to_csv('/drive/My Drive/train.csv', index=False)

## this gives error messages if next parts are done using csv files... so just saved for reference

## Naive Bayes classifier

We build the Naive Bayes classifier from scratch. In order to do this, we create lists of the following: token frequencies in total vocabulary, token frequencies in each genre, number of words in each genre, number of words in total vocabulary, number of total titles, number of titles in each genre. Using these, we compute the priors for each genre and the likelihoods for each word to be in any genre. 
Using both the priors and the likelihoods, we create a dataframe with prediction values. By taking the highest prediction value of a particular word across the genres, we can get the predicted genre for that word.

We then test out whether the model works for the test data.

### Train model: Prep

In [ ]:
from collections import Counter

In [ ]:
# token frequencies within each title (we don't need this...)
unigram_count = []
for title in tokenized_titles:
    uni_title = Counter()
    for i in title:
        uni_title[i] += 1
    unigram_count.append(uni_title)

print(unigram_count[:5]) # test

[Counter({'moms': 1, 'family': 1, 'wall': 1, 'calendar': 1, '2016': 1}), Counter({'doug': 1, 'the': 1, 'pug': 1, '2016': 1, 'wall': 1, 'calendar': 1}), Counter({'moleskine': 1, '2016': 1, 'weekly': 1, 'notebook': 1, '12m': 1, 'large': 1, 'black': 1, 'soft': 1, 'cover': 1, '5': 1, 'x': 1, '825': 1}), Counter({'365': 1, 'cats': 1, 'color': 1, 'pageaday': 1, 'calendar': 1, '2016': 1}), Counter({'sierra': 1, 'club': 1, 'engagement': 1, 'calendar': 1, '2016': 1})]


In [ ]:
# token frequencies in total vocab
tok_freq = Counter()
for title in train['titles']:
    for i in title:
        tok_freq[i] += 1
        
print(tok_freq)

Counter({'the': 92132, 'and': 54409, 'of': 50873, 'a': 31232, 'to': 27842, 'for': 21359, 'in': 21180, 'guide': 15965, 'edition': 12520, 'book': 8121, 'with': 7801, 'your': 7303, 'from': 6510, 'series': 6328, 'on': 5912, 'how': 5902, 'an': 5148, 'new': 4743, 'life': 4710, 'volume': 4018, 'world': 3999, 'history': 3958, 'you': 3875, '1': 3326, 'by': 3190, 'recipes': 3171, 'american': 3028, 'travel': 2949, 'complete': 2761, 'law': 2733, '2': 2689, 'story': 2626, 'art': 2596, 'health': 2551, 'handbook': 2506, 'my': 2491, 'books': 2364, 'science': 2293, 'calendar': 2207, 'at': 2180, 'novel': 2045, 'best': 2042, 'practice': 2013, 'i': 1984, 'guides': 1937, 'cookbook': 1907, 'that': 1888, 'map': 1885, 'vol': 1819, 'love': 1808, 'about': 1783, 'what': 1766, '2016': 1741, 'great': 1738, 'stories': 1685, 'more': 1674, 'war': 1662, 'business': 1625, 'it': 1594, 'is': 1586, 'manual': 1581, 'one': 1566, 'management': 1545, 'modern': 1545, 'introduction': 1539, 'second': 1531, 'other': 1527, 'americ

In [ ]:
# group data by genre
# could probably be done MUCH nicer, but couldn't get for-loop to work...

grouped = train.groupby(train.genres)

Calendars = grouped.get_group("Calendars")
Comics = grouped.get_group("Comics & Graphic Novels")
Test = grouped.get_group("Test Preparation")
Mystery = grouped.get_group("Mystery, Thriller & Suspense")
SciFi = grouped.get_group("Science Fiction & Fantasy")
Romance = grouped.get_group("Romance")
Humor = grouped.get_group("Humor & Entertainment")
Literature = grouped.get_group("Literature & Fiction")
LGBTQ = grouped.get_group("Gay & Lesbian")
Engineering = grouped.get_group("Engineering & Transportation")
Food = grouped.get_group("Cookbooks, Food & Wine")
Crafts = grouped.get_group("Crafts, Hobbies & Home")
Arts = grouped.get_group("Arts & Photography")
Education = grouped.get_group("Education & Teaching")
Parenting = grouped.get_group("Parenting & Relationships")
SelfHelp = grouped.get_group("Self-Help")
Computers = grouped.get_group("Computers & Technology")
Medical = grouped.get_group("Medical Books")
Science = grouped.get_group("Science & Math")
Health = grouped.get_group("Health, Fitness & Dieting")
Business = grouped.get_group("Business & Money")
Law = grouped.get_group("Law")
Biographies = grouped.get_group("Biographies & Memoirs")
History = grouped.get_group("History")
Politics = grouped.get_group("Politics & Social Sciences")
Reference = grouped.get_group("Reference")
Bibles = grouped.get_group("Christian Books & Bibles")
Religion = grouped.get_group("Religion & Spirituality")
Sports = grouped.get_group("Sports & Outdoors")
Teen = grouped.get_group("Teen & Young Adult")
Childrens = grouped.get_group("Children's Books")
Travel = grouped.get_group("Travel")

GenreGroups = [Calendars['titles'], Comics['titles'], Test['titles'], Mystery['titles'], SciFi['titles'], 
               Romance['titles'], Humor['titles'], Literature['titles'], LGBTQ['titles'], Engineering['titles'], 
               Food['titles'], Crafts['titles'], Arts['titles'], Education['titles'], Parenting['titles'], 
               SelfHelp['titles'], Computers['titles'], Medical['titles'], Science['titles'], Health['titles'], 
               Business['titles'], Law['titles'], Biographies['titles'], History['titles'], Politics['titles'], 
               Reference['titles'], Bibles['titles'], Religion['titles'], Sports['titles'], Teen['titles'], 
               Childrens['titles'], Travel['titles']]

In [ ]:
# token frequencies in each genre
genre_count = []
for g in GenreGroups:
    genre_title = Counter()
    for title in g:
        for i in title:
            genre_title[i] += 1
    genre_title = dict(genre_title)
    genre_count.append(genre_title)

In [ ]:
# token frequencies in each genre
genre_count[0] # genres in same order as GenreGroups

{'moms': 12,
 'family': 11,
 'wall': 804,
 'calendar': 1975,
 '2016': 1085,
 'doug': 1,
 'the': 366,
 'pug': 4,
 'moleskine': 28,
 'weekly': 62,
 'notebook': 20,
 '12m': 8,
 'large': 26,
 'black': 27,
 'soft': 9,
 'cover': 30,
 '5': 13,
 'x': 27,
 '825': 12,
 '365': 78,
 'cats': 36,
 'color': 13,
 'pageaday': 56,
 'sierra': 2,
 'club': 3,
 'engagement': 60,
 'wilderness': 6,
 'thomas': 12,
 'kinkade': 9,
 'disney': 22,
 'dreams': 5,
 'collection': 10,
 'ansel': 2,
 'adams': 2,
 'dilbert': 4,
 'daytoday': 95,
 'mary': 7,
 'engelbreit': 4,
 'deluxe': 32,
 'never': 2,
 'give': 1,
 'up': 7,
 'cat': 27,
 'gallery': 11,
 'llewellyns': 6,
 'witches': 3,
 'datebook': 3,
 'amy': 2,
 'knapp': 2,
 'big': 11,
 'grid': 4,
 'essential': 1,
 'organization': 1,
 'and': 148,
 'communication': 1,
 'tool': 1,
 'for': 58,
 'entire': 1,
 'outlander': 1,
 'audubon': 8,
 'nature': 13,
 'national': 13,
 'park': 2,
 'foundation': 1,
 'your': 15,
 'year': 57,
 'mindful': 2,
 'coloring': 12,
 'through': 27,
 'se

In [ ]:
# number of words in a class
len(genre_count[0]) # first genre

2371

In [ ]:
# number of total vocabulary (training set)
V = len(Counter(tok_freq))
print(V)

74712


In [ ]:
# number of titles (in training set)
N_titles = len(train)
print(N_titles)

166088


In [ ]:
# number of titles in each genre
N_genre = train['genres'].value_counts()
print(N_genre[:5])

Travel                       14644
Children's Books             10888
Medical Books                 9642
Health, Fitness & Dieting     9497
Crafts, Hobbies & Home        7983
Name: genres, dtype: int64


### Priors

In [ ]:
# priors of each genre (probability of title being in specific genre)
prob_title = []
for g in range(len(N_genre)):
    prob = N_genre[g] / N_titles
    prob_title.append(prob)

zipped_values = zip(df1.genre.unique(), prob_title)
prior = list(zipped_values)

print(prior[:5]) # test

[('Calendars', 0.08817012667983237), ('Comics & Graphic Novels', 0.06555560907470738), ('Test Preparation', 0.05805356196714995), ('Mystery, Thriller & Suspense', 0.05718053080294783), ('Science Fiction & Fantasy', 0.04806488126776167)]


### Likelihoods

In [ ]:
# likelihoods of each word (probability of word being in specific genre)

# create empty dataframe
likelihood = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq))

len_gc = -1
for i in genre_count: # loop through genres
    len_gc += 1 # create genre index
    for word in i: 
        p = (genre_count[len_gc][word] + 1) / (len(genre_count[len_gc]) + V) # probability
        likelihood.loc[word, likelihood.columns[len_gc]] = p # replace NaN in dataframe with p

In [ ]:
# now fill likelihoods of words that haven't appeared in genre

len_gc = -1
for c, v in likelihood.iteritems(): # loop through genres in dataframe
  len_gc += 1 # create genre index
  p = 1 / (len(genre_count[len_gc]) + V) # probability of word not appearing in that genre
  likelihood[c].fillna(p, inplace=True) # replace remaining NaNs in dataframe with p

In [ ]:
likelihood

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,0.000169,0.000026,0.000013,0.000013,0.000013,0.000013,0.000072,0.000012,0.000039,0.000013,0.000132,0.000035,0.000024,0.000039,0.000317,0.000025,0.000012,0.000012,0.000024,0.000105,0.000036,0.000024,0.000024,0.000012,0.000025,0.000025,0.000084,0.000048,0.000012,0.000024,0.000058,0.000034
family,0.000156,0.000204,0.000206,0.000116,0.000038,0.000391,0.000371,0.000385,0.000477,0.000063,0.002312,0.000363,0.000253,0.000167,0.001573,0.000203,0.000098,0.000557,0.000190,0.001231,0.000655,0.001170,0.000779,0.000356,0.000163,0.000339,0.001059,0.000346,0.000122,0.000314,0.000608,0.000978
wall,0.010443,0.000013,0.000026,0.000052,0.000063,0.000013,0.000263,0.000048,0.000013,0.000038,0.000024,0.000199,0.000302,0.000026,0.000013,0.000013,0.000037,0.000059,0.000119,0.000058,0.000786,0.000073,0.000122,0.000190,0.000038,0.000176,0.000096,0.000179,0.000146,0.000085,0.000210,0.000398
calendar,0.025635,0.000013,0.000013,0.000013,0.000013,0.000025,0.000323,0.000048,0.000013,0.000126,0.000048,0.000258,0.000567,0.000013,0.000038,0.000051,0.000061,0.000012,0.000166,0.000070,0.000060,0.000012,0.000012,0.000012,0.000050,0.000050,0.000108,0.000287,0.000219,0.000012,0.000047,0.000409
2016,0.014089,0.000013,0.001445,0.000013,0.000025,0.000013,0.000311,0.000084,0.000026,0.000390,0.000060,0.000410,0.000555,0.000245,0.000089,0.000089,0.000539,0.000604,0.000142,0.000093,0.000357,0.000134,0.000012,0.000024,0.000063,0.000364,0.000169,0.000346,0.000219,0.000012,0.000129,0.001354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50mile,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
flashmaps,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
ballrooms,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
riverboat,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023


In [ ]:
# predictions on word by word basis using words in titles of test data

predictors = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq)) # create empty dataframe
len_gc = -1
for i in prior:
  len_gc += 1 # create genre index
  # square all likelihoods then multiply them each with prior of that genre
  predictors = likelihood.loc[:, df1.genre.unique().tolist()] ** 2 # square likelihoods
  predictors = predictors.loc[:, df1.genre.unique().tolist()] * np.array(prior[len_gc][1]) # multiply by priors

In [ ]:
predictors

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,1.859778e-10,4.255846e-12,1.088734e-12,1.085314e-12,1.040077e-12,1.038531e-12,3.364692e-11,9.479630e-13,9.778155e-12,1.033914e-12,1.147035e-10,8.085061e-12,3.807974e-12,9.753497e-12,6.578407e-10,4.204524e-12,9.821680e-13,9.180544e-13,3.685044e-12,7.142483e-11,8.336212e-12,3.884462e-12,3.877088e-12,9.208893e-13,4.086996e-12,4.122580e-12,4.641330e-11,1.493382e-11,9.662344e-13,3.813126e-12,2.233320e-11,7.613570e-12
family,1.584663e-10,2.723741e-10,2.787159e-10,8.791045e-11,9.360697e-12,9.980286e-10,8.981857e-10,9.707141e-10,1.487366e-09,2.584784e-11,3.494571e-08,8.633048e-10,4.198292e-10,1.831490e-10,1.618393e-08,2.690896e-10,6.285875e-11,2.027982e-09,2.358428e-10,9.907771e-09,2.801893e-09,8.949800e-09,3.970138e-09,8.288004e-10,1.726756e-10,7.513402e-10,7.335195e-09,7.849591e-10,9.662344e-11,6.444182e-10,2.415559e-09,6.256663e-09
wall,7.131257e-07,1.063962e-12,4.354936e-12,1.736503e-11,2.600194e-11,1.038531e-12,4.523641e-10,1.516741e-11,1.086462e-12,9.305222e-12,3.791852e-12,2.596203e-10,5.949960e-10,4.334888e-12,1.052545e-12,1.051131e-12,8.839512e-12,2.295136e-11,9.212610e-11,2.204470e-11,4.034726e-09,3.496016e-11,9.692719e-11,2.357477e-10,9.195740e-12,2.020064e-10,6.062145e-11,2.100069e-10,1.391377e-10,4.671079e-11,2.894383e-10,1.036292e-09
calendar,4.296831e-06,1.063962e-12,1.088734e-12,1.085314e-12,1.040077e-12,4.154125e-12,6.813500e-10,1.516741e-11,1.086462e-12,1.033914e-10,1.516741e-11,4.347966e-10,2.102954e-09,1.083722e-12,9.472907e-12,1.681810e-11,2.455420e-11,9.180544e-13,1.805672e-10,3.174437e-11,2.315614e-11,9.711155e-13,9.692719e-13,9.208893e-13,1.634798e-11,1.649032e-11,7.672402e-11,5.376176e-10,3.130599e-10,9.532814e-13,1.429325e-11,1.096354e-09
2016,1.297878e-06,1.063962e-12,1.365708e-08,1.085314e-12,4.160310e-12,1.038531e-12,6.318143e-10,4.645019e-11,4.345847e-12,9.935909e-10,2.369907e-11,1.100467e-09,2.014418e-09,3.912236e-10,5.157471e-11,5.150542e-11,1.901477e-09,2.387860e-09,1.326616e-10,5.643444e-11,8.336212e-10,1.175050e-10,9.692719e-13,3.683557e-12,2.554372e-11,8.667725e-10,1.856532e-10,7.849591e-10,3.130599e-10,9.532814e-13,1.080927e-10,1.197953e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50mile,1.100460e-12,1.063962e-12,1.088734e-12,1.085314e-12,1.040077e-12,1.038531e-12,9.346365e-13,9.479630e-13,1.086462e-12,1.033914e-12,9.479630e-13,8.983401e-13,9.519936e-13,1.083722e-12,1.052545e-12,1.051131e-12,9.821680e-13,9.180544e-13,9.212610e-13,8.817881e-13,9.262457e-13,9.711155e-13,9.692719e-13,9.208893e-13,1.021749e-12,1.030645e-12,9.472101e-13,9.333640e-13,9.662344e-13,9.532814e-13,8.933280e-13,3.383809e-12
flashmaps,1.100460e-12,1.063962e-12,1.088734e-12,1.085314e-12,1.040077e-12,1.038531e-12,9.346365e-13,9.479630e-13,1.086462e-12,1.033914e-12,9.479630e-13,8.983401e-13,9.519936e-13,1.083722e-12,1.052545e-12,1.051131e-12,9.821680e-13,9.180544e-13,9.212610e-13,8.817881e-13,9.262457e-13,9.711155e-13,9.692719e-13,9.208893e-13,1.021749e-12,1.030645e-12,9.472101e-13,9.333640e-13,9.662344e-13,9.532814e-13,8.933280e-13,3.383809e-12
ballrooms,1.100460e-12,1.063962e-12,1.088734e-12,1.085314e-12,1.040077e-12,1.038531e-12,9.346365e-13,9.479630e-13,1.086462e-12,1.033914e-12,9.479630e-13,8.983401e-13,9.519936e-13,1.083722e-12,1.052545e-12,1.051131e-12,9.821680e-13,9.180544e-13,9.212610e-13,8.817881e-13,9.262457e-13,9.711155e-13,9.692

In [ ]:
predictors.loc[["moms"]]

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,1.859778e-10,4.255846e-12,1.088734e-12,1.085314e-12,1.040077e-12,1.038531e-12,3.364692e-11,9.479630e-13,9.778155e-12,1.033914e-12,1.147035e-10,8.085061e-12,3.807974e-12,9.753497e-12,6.578407e-10,4.204524e-12,9.821680e-13,9.180544e-13,3.685044e-12,7.142483e-11,8.336212e-12,3.884462e-12,3.877088e-12,9.208893e-13,4.086996e-12,4.122580e-12,4.641330e-11,1.493382e-11,9.662344e-13,3.813126e-12,2.233320e-11,7.613570e-12


In [ ]:
# save predictors so these don't have to be created again when testing model
predictors.to_csv('/drive/My Drive/predictors.csv', index=False)

In [ ]:
# output genre that has highest prediction value for input word
maxValueIndexObj = predictors.idxmax(axis=1)
print(list(maxValueIndexObj[["high"]]))
print(list(maxValueIndexObj[["higher"]]))
print(list(maxValueIndexObj[["highest"]]))

['Health, Fitness & Dieting']
['Education & Teaching']
['Sports & Outdoors']


### Test model

In [ ]:
# function that will return list of predicted genres based on title input
# function is very slow -> OPTIMIZE (otherwise takes 11.5hrs for test dataset...)
def pred_genre(title):
  prediction = []
  for w in title:
    ## CONDITION: if word exists in total vocabulary
    if w in predictors.index:
      maxValueIndexObj = predictors.idxmax(axis=1)
      prediction += list(maxValueIndexObj[[w]])
    ## CONDITION: if word does not exist in total vocabulary
    if w not in predictors.index:
      pass

  # probability of predicted genre being chosen
  d = dict(Counter(prediction))
  d1 = {k: v / total for total in (sum(d.values()),) for k, v in d.items()}

  # output most likely predicted genre
  if prediction == []:
    return 'There is no prediction for this title.'
  else:
    return max(d1, key=d1.get) 
  
  # what if there is no maximum? (all equally predicted)
  # gives first predicted genre as output...

In [ ]:
# predicted genre for first test title
import time
time_start = time.clock()

pred_genre(test['titles'][0])

time_elapsed = (time.clock() - time_start)
print(time_elapsed)

0.8072289999999995


In [ ]:
pred_genre(test['titles'][678])

'There is no prediction for this title.'

In [ ]:
# "real" genre of first test title
test['genres'][0]

'Calendars'

In [ ]:
# titles that would give no prediction:
# these titles are one word long and not included in training set
for t in test['titles']:
  if len(t) == 1:
    for w in t:
      if w not in list(tok_freq.keys()):
        print(t)
  else:
    pass

['xenozoic']
['daytripper']
['unflattening']
['dolltopia']
['verboten']
['shortcomings']
['spiderverse']
['sofrito']
['swipe']
['trapeze']
['superposition']
['darknet']
['agroland']
['slab']
['abilites']
['tigerman']
['statisticity']
['vermilion']
['thugalicious']
['caparazones']
['queerwolf']
['pybrac']
['edenbrooke']
['lespada']
['modogamous']
['snowmageddon']
['jolt']
['apolonia']
['genafinn']
['pornstar']
['haywire']
['beatlesongs']
['dvf']
['steamfunk']
['horsedreams']
['olio']
['shibumi']
['mathesis']
['trainspotting']
['queerspawn']
['outsight']
['maina']
['calloustown']
['habibi']
['kirakira']
['wozzeck']
['bodymap']
['nancore']
['quesadillas']
['burritos']
['panerai']
['palio']
['autoprogettazione']
['quaintance']
['instamatics']
['strapless']
['unbranded']
['zendoodles']
['doodleology']
['aeromonas']
['diseaseproof']
['schistosomiasis']
['muslin']
['specks']
['dreamseller']
['lifesmart']
['breatheology']
['insomniac']
['stotan']
['psychward']
['disneywar']
['getitsettled']
['

In [ ]:
######## this function takes 9.25 hours!!
predicted_genre = []
for t in test['titles']:
  predicted_genre.append(pred_genre(t))


# change list to dataframe to enable saving as csv file
df2 = pd.DataFrame(predicted_genre)
df2.columns = ['Predicted Genre']
df2['True Genre'] = test['genres']

# create and save csv file in Google Drive
df2.to_csv('/drive/My Drive/predicted_genre.csv', index=False)

In [ ]:
# read csv file as pandas Dataframe
predicted_genre = pd.read_csv('/drive/My Drive/predicted_genre.csv',encoding='latin1')

In [ ]:
# dataframe with predicted and true genres
predicted_genre

,Predicted Genre,True Genre
0,Calendars,Calendars
1,Calendars,Calendars
2,Calendars,Calendars
3,Calendars,Calendars
4,Children's Books,Calendars
...,...,...
41479,Travel,Travel
41480,Travel,Travel
41481,Travel,Travel
41482,Travel,Travel


### Accuracy of model

In [ ]:
# read csv file as pandas Dataframe
predicted_genre = pd.read_csv('/drive/My Drive/predicted_genre.csv',encoding='latin1')

In [ ]:
# compute overall accuracy, precision, recall, f1 scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Accuracy: ', accuracy_score(predicted_genre['True Genre'], predicted_genre['Predicted Genre']))
print('Precision: ', precision_score(predicted_genre['True Genre'], predicted_genre['Predicted Genre'], average='weighted'))
print('Recall: ', recall_score(predicted_genre['True Genre'], predicted_genre['Predicted Genre'], average='weighted', zero_division=1))
print('F1:', f1_score(predicted_genre['True Genre'], predicted_genre['Predicted Genre'], average='weighted'))

Accuracy:  0.4342156011956417
Precision:  0.5212644190741561
Recall:  0.4342156011956417
F1: 0.42028788747206675


In [ ]:
# compute accuracy, precision, recall, f1 scores by genre

from sklearn.metrics import precision_recall_fscore_support as score

# precision, recall, fscore, support separated by genre
precision, recall, fscore, support = score(predicted_genre['True Genre'], predicted_genre['Predicted Genre'])

df_acc = pd.DataFrame()
df_acc['precision']=pd.Series(precision)
df_acc['recall']=pd.Series(recall)
df_acc['fscore']=pd.Series(fscore)
df_acc['support']=pd.Series(support)

index = list(predicted_genre['True Genre'].unique())
index.insert(31, 'no prediction')
df_acc.index = index

print(df_acc)

                              precision    recall    fscore  support
Calendars                      0.518919  0.228390  0.317181     1261
Comics & Graphic Novels        0.238494  0.067696  0.105458      842
Test Preparation               0.570093  0.457271  0.507488     2001
Mystery, Thriller & Suspense   0.827655  0.782197  0.804284      528
Science Fiction & Fantasy      0.442694  0.486198  0.463427     2717
Romance                        0.529506  0.379429  0.442077     1750
Humor & Entertainment          0.685315  0.313099  0.429825      626
Literature & Fiction           0.708529  0.685956  0.697060     1538
Gay & Lesbian                  0.690090  0.652100  0.670557     1762
Engineering & Transportation   0.660951  0.434649  0.524428     1951
Cookbooks, Food & Wine         0.573333  0.134375  0.217722      320
Crafts, Hobbies & Home         0.677419  0.233766  0.347586      539
Arts & Photography             0.482759  0.110672  0.180064      253
Education & Teaching           0.2

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# save model evaluation as csv file in Google Drive
df_acc.to_csv('/drive/My Drive/NB_model_eval.csv')